In [70]:
from sqlalchemy import create_engine, text

# DB 연결 설정
user = "postgres"
password = "0000"
host = "localhost"
port = "5432"
db_name = "rrt"

engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db_name}")

with engine.connect() as conn:
    result = conn.execute(text("SELECT current_database();"))  # ← 여기!
    print("Connected to:", result.scalar())

In [44]:
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd
import re

dir_path = Path("../data/sample/Bloomberg/SPX_12232024/")
files = [f for f in dir_path.glob("*.xlsx") if not f.name.startswith("~$") and not f.name.startswith("ATMs")]
files.sort()

for f in files:
    print(f.name)

In [6]:
valid_call

In [ ]:
S0 = 5975
r = 

In [11]:
dir_path.

In [63]:
atm = pd.read_excel("../data/sample/Bloomberg/SPX_12232024/ATMs.xlsx", engine="openpyxl", header=None).dropna()
atm.columns = atm.iloc[0, :]
atm = atm.drop(atm.index[0])
call_atm = atm.iloc[:, :15]
put_atm = atm.iloc[:, 15:]
cols = [
    'strike',
    'ticker',
    'bid',
    'ask',
    'last_p',
    'iv',
    'vol',
    'div',
    'ndvs',
    'tau',
    'csize',
    'r',
    'hv30',
    'hv60',
    'hv90'
]
call_atm.columns = cols
call_atm.loc[:, ['refdate']] = datetime(2024, 12, 23)
call_atm.loc[:, ['cp_flag']] = 'C'
put_atm.columns = cols
put_atm.loc[:, ['refdate']] = datetime(2024, 12, 23)
put_atm.loc[:, ['cp_flag']] = 'P'

atm_all = pd.concat([call_atm, put_atm], ignore_index=True, axis=0)
print(atm_all.shape)

In [75]:
with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS atms;"))

with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE atms (
            strike     NUMERIC,
            ticker     TEXT NOT NULL,
            bid        NUMERIC,
            ask        NUMERIC,
            last_p     NUMERIC,
            iv         NUMERIC,
            vol        NUMERIC,
            div        NUMERIC,
            ndvs       NUMERIC,
            tau        NUMERIC,
            csize      NUMERIC,
            r          NUMERIC,
            hv30       NUMERIC,
            hv60       NUMERIC,
            hv90       NUMERIC,
            refdate    DATE NOT NULL,
            cp_flag    CHAR(1),
            PRIMARY KEY (refdate, ticker)
        )
    """))

with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT
            tc.constraint_name,
            tc.table_name,
            kcu.column_name
        FROM 
            information_schema.table_constraints AS tc
        JOIN 
            information_schema.key_column_usage AS kcu
            ON tc.constraint_name = kcu.constraint_name
        WHERE 
            tc.table_name = 'atms'
            AND tc.constraint_type = 'PRIMARY KEY';
    """))
    pk_info = result.fetchall()
    print("Primary Key columns in 'atm':", [row[2] for row in pk_info])
    
atm_all.to_sql("atm", con=engine, if_exists='append', index=False, method='multi')
print("successfully inserted atm table")
df = pd.read_sql("SELECT * FROM atm", con=engine)
display(df)

In [62]:
df[df.duplicated()]

In [76]:
import pandas as pd
import re
from pathlib import Path

# 옵션 컬럼 정의
cols = [
    'ticker',  # Option ticker (e.g., "SPXW 12/31/24 C1200")
    'strike',  # Strike price of the option
    'bid',     # Best bid price (buying offer)
    'ask',     # Best ask price (selling offer)
    'last',    # Last traded price
    'iv',      # Implied volatility (IV)
    'volume',  # Trading volume
    'open',    # Open interest
    'delta',   # Delta
    'gamma',   # Gamma
    'theta',   # Theta
    'vega',    # Vega
    'rho',     # Rho
    'tau',     # Time to expiration
    'option_type',    # Option type: "Euro" or "American"
    'thiv',    # Theoretical IV
    'thpr'     # Theoretical price
]

underlying_tickers = ['SPX', 'SPXW']
ticker_pattern = r'^(' + '|'.join(underlying_tickers) + r')\d{1,2}/\d{1,2}/\d{2,4}[CP]\d+(\.\d+)?$'

for f in files:
    
    df = pd.read_excel(f, engine="openpyxl", header=None)
    half = df.shape[1] // 2
    
    df_call = df.iloc[:, :half].copy()
    df_call.columns = cols
    df_call["type"] = "C"
    mask_call = (
        df_call["ticker"]
        .astype(str)
        .apply(lambda x: re.sub(r"\s+", "", x))
        .str.match(ticker_pattern)
    )
    # a = df_call["ticker"].astype(str).apply(lambda x: re.sub(r"\s+", "", x))
    # b = df_call["ticker"].astype(str).apply(lambda x: re.sub(r"\s+", "", x)).str.match(ticker_pattern)
    # display(pd.concat([a, mask_call], axis=1))
    invalid_call = df_call[~mask_call]
    valid_call = df_call[mask_call]

    df_put = df.iloc[:, half:].copy()
    df_put.columns = cols
    df_put["type"] = "P"
    mask_put = (
        df_put["ticker"]
        .astype(str)
        .apply(lambda x: re.sub(r"\s+", "", x))
        .str.match(ticker_pattern)
    )
    invalid_put = df_put[~mask_put]
    valid_put = df_put[mask_put]

In [77]:
valid_call

In [13]:
pd.read_excel('../data/sample/raw_data/SPX_12232024/20251017.xlsx')

In [16]:
from pathlib import Path

print("현재 작업 디렉토리:", Path.cwd())

In [41]:
import pandas as pd
pd.read_excel("../data/sample/raw_data/SPX_12232024/" + "20250117.xlsx", engine='openpyxl')

In [12]:
!pip install openpyxl